In [1]:
import numpy as np
import pandas as pd

pd.options.display.max_columns = 8
pd.options.display.max_rows = 8

In [2]:
## P3
df = pd.read_csv('adult.data', header=None)
df.columns = ['age', 'workclass', 'fnlwgt', 'education', 'education-num',
              'marital-status', 'occupation', 'relationship', 'race', 'sexz',
              'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
              'income']
df

,age,workclass,fnlwgt,education,...,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,...,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,...,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,...,0,40,United-States,<=50K
3,53,Private,234721,11th,...,0,40,United-States,<=50K
...,...,...,...,...,...,...,...,...,...
32557,40,Private,154374,HS-grad,...,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,...,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,...,0,20,United-States,<=50K
32560,52,Self-emp-inc,287927,HS-grad,...,0,40,United-States,>50K


In [3]:
df[['age', 'marital-status']]

,age,marital-status
0,39,Never-married
1,50,Married-civ-spouse
2,38,Divorced
3,53,Married-civ-spouse
...,...,...
32557,40,Married-civ-spouse
32558,58,Widowed
32559,22,Never-married
32560,52,Married-civ-spouse


In [4]:
df.groupby('income')['hours-per-week'].mean().to_frame()

,hours-per-week
income,
<=50K,38.840210
>50K,45.473026


In [5]:
# NumPy
arr = np.array([1, 2, 3, 4], dtype=np.int64)
arr

array([1, 2, 3, 4])

In [6]:
pd.DataFrame(arr).T

,0,1,2,3
0,1,2,3,4


## pandas Development

In [7]:
df = pd.DataFrame({'X': [1, 2, 3], 'Y': [4, 5, 6],
                   'Z': [True, False, True]}, index=['a', 'b', 'c'])
df

,X,Y,Z
a,1,4,True
b,2,5,False
c,3,6,True


In [8]:
df.reindex(['b', 'a', 'c'])

,X,Y,Z
b,2,5,False
a,1,4,True
c,3,6,True


In [9]:
df.index.get_indexer(['b', 'a', 'c'])

array([1, 0, 2])

In [10]:
df.values

array([[1, 4, True],
       [2, 5, False],
       [3, 6, True]], dtype=object)

In [11]:
np.take(df.values, df.index.get_indexer(['b', 'a', 'c']), axis=0)

array([[2, 5, False],
       [1, 4, True],
       [3, 6, True]], dtype=object)

## Performance 

### Environment setup

In [12]:
import numpy.distutils.system_info as sysinfo

In [13]:
sysinfo.get_info('lapack')

{'language': 'f77',
 'libraries': ['openblas'],
 'library_dirs': ['/home/ec2-user/miniconda/lib']}

In [14]:
sysinfo.get_info('atlas')

{'define_macros': [('ATLAS_INFO', '"\\"3.8.4\\""')],
 'include_dirs': ['/home/ec2-user/miniconda/include'],
 'language': 'f77',
 'libraries': ['lapack', 'f77blas', 'cblas', 'atlas'],
 'library_dirs': ['/home/ec2-user/miniconda/lib']}

In [15]:
pd.show_versions()


INSTALLED VERSIONS
------------------
commit: None
python: 2.7.10.final.0
python-bits: 64
OS: Linux
OS-release: 4.1.7-15.23.amzn1.x86_64
machine: x86_64
processor: x86_64
byteorder: little
LC_ALL: None
LANG: ja_JP.UTF-8

pandas: 0.16.2
nose: 1.3.7
Cython: 0.23.3
numpy: 1.9.3
scipy: 0.16.0
statsmodels: 0.6.1
IPython: 4.0.0
sphinx: None
patsy: 0.4.0
dateutil: 2.4.2
pytz: 2015.6
bottleneck: None
tables: None
numexpr: None
matplotlib: 1.4.3
openpyxl: None
xlrd: None
xlwt: None
xlsxwriter: None
lxml: None
bs4: None
html5lib: None
httplib2: None
apiclient: None
sqlalchemy: None
pymysql: None
psycopg2: None


In [22]:
import pandas.util.testing as tm
np.random.seed(1)

chars1 = tm.rands_array(5, 100)
chars2 = tm.rands_array(5, 10000)

N = 1000000

df = pd.DataFrame({'a': np.random.randn(N),
                   'b': tm.choice(chars1, size=N),
                   'c': tm.choice(chars2, size=N),
                   'd': np.random.randint(1, 1000, N)})
df.shape

(1000000, 4)

In [23]:
df.head()

,a,b,c,d
0,-0.009448,jWCZV,LCFyX,612
1,-0.086430,mYMBW,hOALb,893
2,0.145731,pXxGQ,5PtON,754
3,0.710000,pXxGQ,IYF4L,492
4,-2.217829,PaSty,n5ijN,777


### Use built-in

In [20]:
def f1(s):
    return s['b'] + s['c']

%timeit df.apply(f1, axis=1)

1 loops, best of 3: 12 s per loop


In [21]:
%timeit df['b'] + df['c']

1000 loops, best of 3: 1.33 ms per loop


In [24]:
def f2_1(s):
    if s['a'] > 0:
        return s['b'] + s['c']
    else:
        return s['c'] + s['b']

%timeit df.apply(f2_1, axis=1)

1 loops, best of 3: 16 s per loop


In [26]:
def f2_2(a, b, c):
    if a > 0:
        return b + c
    else:
        return c + b

%timeit pd.Series(np.vectorize(f2_2)(df['a'], df['b'], df['c']), index=df.index)

1 loops, best of 3: 368 ms per loop


In [30]:
%timeit (df['b'] + df['c']).where(df['a'] > 0, df['c'] + df['b'])

1 loops, best of 3: 200 ms per loop


In [27]:
%load_ext cython

In [28]:
%%cython

import numpy as np
from numpy cimport ndarray

def f2_3(ndarray[double, ndim=1] a,
         ndarray[object, ndim=1] b,
         ndarray[object, ndim=1] c):
    
    cdef:
        int i, length = len(a)
        double aval
        object bval, cval
        ndarray[object, ndim=1] result = np.empty(length, dtype=object)
        
    for i in range(length):
        aval = a[i]
        bval = b[i]
        cval = c[i]
        if aval > 0:
            result[i] = bval + cval
        else:
            result[i] = cval + bval
            
    return result

In [29]:
%timeit pd.Series(f2_3(df['a'].values, df['b'].values, df['c'].values), index=df.index)

10 loops, best of 3: 68.7 ms per loop


In [ ]:
# Numba

### pandas hash functions may faster than NumPy

In [28]:
%timeit np.unique(df['d'])

10 loops, best of 3: 43.5 ms per loop


In [29]:
%timeit pd.unique(df['d'])

100 loops, best of 3: 5.97 ms per loop


### Use single op as much 

In [30]:
# silly example...
%timeit df['a'].add(1).sub(2)

1000 loops, best of 3: 1.77 ms per loop


In [31]:
%timeit df['a'].sub(1)

1000 loops, best of 3: 688 µs per loop


### Avoid object, use Categorical

In [32]:
df.dtypes

a    float64
b     object
c     object
d      int64
dtype: object

In [33]:
df.memory_usage()

a    8000000
b    8000000
c    8000000
d    8000000
dtype: int64

In [34]:
%timeit df.groupby('b').mean()

10 loops, best of 3: 52.1 ms per loop


In [35]:
df['b'] = df['b'].astype('category')
# also good for memory
df.memory_usage()

a    8000000
b    1000800
c    8000000
d    8000000
dtype: int64

In [36]:
%timeit df.groupby('b').mean()

10 loops, best of 3: 27 ms per loop


In [37]:
df['b'].values

[jWCZV, mYMBW, pXxGQ, pXxGQ, PaSty, ..., ngzMW, EGwnP, 5tzXo, ngzMW, J1VU8]
Length: 1000000
Categories (100, object): [1jP5y, 2GPsw, 2pfrQ, 3sFCE, ..., yx3bX, z1u56, zBRbs, zVIxh]

### Use sorted / unique Index

In [38]:
%timeit df.join(df, rsuffix='right_')

10 loops, best of 3: 52.2 ms per loop


In [39]:
df2 = df.sample(n=len(df))
%timeit df2.join(df, rsuffix='right_')

1 loops, best of 3: 173 ms per loop


In [40]:
df.index.is_unique

True

In [41]:
df.index.is_monotonic_increasing

True

### Date parsing

In [31]:
iso_8641_fmt = '2011-{0:02d}-{1:02d} 00:00:00'
[iso_8641_fmt.format(m, d) for m, d in zip([1, 2], [3, 4])]

['2011-01-03 00:00:00', '2011-02-04 00:00:00']

In [32]:
pd.to_datetime([iso_8641_fmt.format(m, d) for m, d in zip([1, 2], [3, 4])])

DatetimeIndex(['2011-01-03', '2011-02-04'], dtype='datetime64[ns]', freq=None, tz=None)

In [33]:
mdy_fmt = '{0:02d}/{1:02d}/2011'
[mdy_fmt.format(m, d) for m, d in zip([1, 2], [3, 4])]

['01/03/2011', '02/04/2011']

In [34]:
pd.to_datetime([mdy_fmt.format(m, d) for m, d in zip([1, 2], [3, 4])])

DatetimeIndex(['2011-01-03', '2011-02-04'], dtype='datetime64[ns]', freq=None, tz=None)

In [ ]:
N = 1000000

months = np.random.randint(1, 12, N)
days = np.random.randint(1, 28, N)
dates = [iso_8641_fmt.format(m, d) for m, d in zip(months, days)]
%timeit pd.to_datetime(dates)

10 loops, best of 3: 175 ms per loop


In [ ]:
dates = [mdy_fmt.format(m, d) for m, d in zip(months, days)]
%timeit pd.to_datetime(dates)